In [25]:
# Add these imports at the top of your notebook
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor, VotingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings('ignore')



In [26]:
def create_modeling_pipeline(data):
    """
    Complete pipeline for real estate price prediction with log transformation
    """
    
    if 'price' not in data.columns:
        raise ValueError("'price' column not found in data")
    
    X = data.drop('price', axis=1)
    y = data['price']
    
    # Log transform target and features
    y = np.log1p(y)
    numerical_features = X.select_dtypes(include=['float64']).columns.tolist()
    for col in numerical_features:
        X[col] = np.log1p(X[col])
    
    categorical_features = X.select_dtypes(include=['int64']).columns.tolist()
    
    print(f"Number of features: {X.shape[1]}")
    print(f"Categorical features: {len(categorical_features)}")
    print(f"Numerical features: {len(numerical_features)}")
    print(f"Total samples: {X.shape[0]}")
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=True
    )
    
    # Original y for metrics
    original_y_train = np.expm1(y_train)
    original_y_test = np.expm1(y_test)
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', RobustScaler(), numerical_features),
            ('cat', 'passthrough', categorical_features)
        ])
    
    # Models with increased complexity for trees
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(alpha=1.0, random_state=42),
        'Lasso Regression': Lasso(alpha=0.1, random_state=42),
        'ElasticNet': ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42),
        'Random Forest': RandomForestRegressor(
            n_estimators=200, 
            max_depth=None, 
            random_state=42,
            n_jobs=-1
        ),
        'Gradient Boosting': GradientBoostingRegressor(
            n_estimators=200, 
            learning_rate=0.1,
            max_depth=7,
            random_state=42
        ),
        'XGBoost': xgb.XGBRegressor(
            n_estimators=200,
            max_depth=8,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            n_jobs=-1,
            verbosity=0
        ),
        'LightGBM': lgb.LGBMRegressor(
            n_estimators=200,
            max_depth=9,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            n_jobs=-1,
            verbose=-1
        ),
        'CatBoost': CatBoostRegressor(
            iterations=200,
            depth=8,
            learning_rate=0.1,
            random_seed=42,
            verbose=0,
            cat_features=categorical_features if categorical_features else None
        ),
        'K-Nearest Neighbors': KNeighborsRegressor(n_neighbors=5, n_jobs=-1),
        'Support Vector Regression': SVR(kernel='linear', C=1.0, epsilon=0.1),  # Changed to linear for speed
        'Kernel Ridge Regression': KernelRidge(alpha=1.0, kernel='polynomial', degree=3),
    }
    
    pipelines = {}
    for name, model in models.items():
        pipelines[name] = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])
    
    # Ensembles with updated base models
    estimators = [
        ('xgb', xgb.XGBRegressor(n_estimators=100, max_depth=6, random_state=42, verbosity=0)),
        ('lgbm', lgb.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)),
        ('rf', RandomForestRegressor(n_estimators=100, random_state=42))
    ]
    
    stacking_reg = StackingRegressor(
        estimators=estimators,
        final_estimator=Ridge(alpha=1.0),
        cv=3
    )
    
    pipelines['Stacking Ensemble'] = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', stacking_reg)
    ])
    
    voting_reg = VotingRegressor([
        ('xgb', xgb.XGBRegressor(n_estimators=100, random_state=42, verbosity=0)),
        ('lgbm', lgb.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)),
        ('rf', RandomForestRegressor(n_estimators=100, random_state=42))
    ])
    
    pipelines['Voting Ensemble'] = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', voting_reg)
    ])
    
    return X_train, X_test, y_train, y_test, original_y_train, original_y_test, pipelines

In [27]:
def evaluate_models(pipelines, X_train, X_test, y_train, y_test, original_y_train, original_y_test):
    """
    Evaluate all models with metrics on original scale
    """
    results = []
    feature_importances = {}
    
    print("=" * 80)
    print("MODEL COMPARISON AND EVALUATION")
    print("=" * 80)
    
    for name, pipeline in pipelines.items():
        print(f"\n{'='*60}")
        print(f"Training {name}...")
        print('='*60)
        
        try:
            pipeline.fit(X_train, y_train)
            
            # Predict and back-transform
            y_pred_train_log = pipeline.predict(X_train)
            y_pred_test_log = pipeline.predict(X_test)
            y_pred_train = np.expm1(y_pred_train_log)
            y_pred_test = np.expm1(y_pred_test_log)
            
            # Metrics on original scale
            metrics = {
                'Model': name,
                'Train RMSE': np.sqrt(mean_squared_error(original_y_train, y_pred_train)),
                'Test RMSE': np.sqrt(mean_squared_error(original_y_test, y_pred_test)),
                'Train MAE': mean_absolute_error(original_y_train, y_pred_train),
                'Test MAE': mean_absolute_error(original_y_test, y_pred_test),
                'Train R²': r2_score(original_y_train, y_pred_train),
                'Test R²': r2_score(original_y_test, y_pred_test),
                'Test MAPE': mean_absolute_percentage_error(original_y_test, y_pred_test)
            }
            
            # CV on log scale, but can adjust if needed
            cv_scores = cross_val_score(pipeline, X_train, y_train, 
                                        cv=5, scoring='neg_root_mean_squared_error')
            metrics['CV RMSE (Mean)'] = -cv_scores.mean()  # on log scale
            metrics['CV RMSE (Std)'] = cv_scores.std()
            
            results.append(metrics)
            
            print(f"\n{name} Performance (original scale):")
            print(f"  Test RMSE: {metrics['Test RMSE']:.2f}")
            print(f"  Test MAE: {metrics['Test MAE']:.2f}")
            print(f"  Test R²: {metrics['Test R²']:.3f}")
            print(f"  Cross-validation RMSE (log scale): {metrics['CV RMSE (Mean)']:.2f} (+/- {metrics['CV RMSE (Std)']:.2f})")
            
            if hasattr(pipeline.named_steps['model'], 'feature_importances_'):
                importances = pipeline.named_steps['model'].feature_importances_
                if hasattr(pipeline.named_steps['preprocessor'], 'get_feature_names_out'):
                    feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()
                else:
                    feature_names = X_train.columns
                
                feature_importances[name] = pd.DataFrame({
                    'feature': feature_names,
                    'importance': importances
                }).sort_values('importance', ascending=False)
                
                print(f"\n  Top 5 Important Features:")
                for i, (feat, imp) in enumerate(feature_importances[name].head().values):
                    print(f"    {i+1}. {feat}: {imp:.4f}")
            
        except Exception as e:
            print(f"Error training {name}: {str(e)}")
            continue
    
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('Test RMSE')
    
    print("\n" + "="*80)
    print("MODEL COMPARISON SUMMARY (Sorted by Test RMSE)")
    print("="*80)
    display_cols = ['Model', 'Test RMSE', 'Test MAE', 'Test R²', 'CV RMSE (Mean)', 'CV RMSE (Std)']
    print(results_df[display_cols].to_string(index=False))
    
    return results_df, feature_importances

In [28]:
def hyperparameter_tuning(pipelines, X_train, y_train, model_name='LightGBM'):
    """
    Perform hyperparameter tuning for a specific model
    """
    if model_name not in pipelines:
        print(f"Model {model_name} not found in pipelines")
        return None
    
    print(f"\n{'='*60}")
    print(f"HYPERPARAMETER TUNING FOR {model_name.upper()}")
    print('='*60)
    
    # Define parameter grids for different models
    param_grids = {
        'LightGBM': {
            'model__n_estimators': [100, 200, 300, 500, 1000],
            'model__max_depth': [3, 5, 7, 9, 11, -1],
            'model__learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
            'model__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
            'model__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
            'model__reg_alpha': [0, 0.001, 0.01, 0.1, 0.5, 1],
            'model__reg_lambda': [0, 0.001, 0.01, 0.1, 0.5, 1],
            'model__num_leaves': [31, 63, 127, 255]
        },
        'XGBoost': {
            'model__n_estimators': [100, 200, 300, 500],
            'model__max_depth': [3, 5, 7, 9, 11],
            'model__learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
            'model__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
            'model__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
            'model__gamma': [0, 0.1, 0.2, 0.3, 0.4],
            'model__reg_alpha': [0, 0.001, 0.01, 0.1],
            'model__reg_lambda': [0.5, 1, 1.5, 2]
        },
        'Random Forest': {
            'model__n_estimators': [50, 100, 200, 300],
            'model__max_depth': [5, 10, 15, 20, None],
            'model__min_samples_split': [2, 5, 10, 20],
            'model__min_samples_leaf': [1, 2, 4, 8],
            'model__max_features': ['sqrt', 'log2', None, 0.5, 0.7]
        },
        'CatBoost': {
            'model__iterations': [100, 200, 300, 500],
            'model__depth': [4, 6, 8, 10, 12],
            'model__learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
            'model__l2_leaf_reg': [1, 3, 5, 7, 9],
            'model__subsample': [0.6, 0.8, 1.0],
            'model__colsample_bylevel': [0.6, 0.8, 1.0]
        },
        'Gradient Boosting': {
            'model__n_estimators': [100, 200, 300, 500],
            'model__learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
            'model__max_depth': [3, 5, 7, 9],
            'model__subsample': [0.6, 0.8, 1.0],
            'model__min_samples_split': [2, 5, 10],
            'model__min_samples_leaf': [1, 2, 4],
            'model__max_features': ['sqrt', 'log2', None]
        },
        'Support Vector Regression': {
            'model__kernel': ['linear', 'rbf'],
            'model__C': [0.1, 1, 10, 100],
            'model__epsilon': [0.01, 0.1, 0.5, 1],
            'model__gamma': ['scale', 'auto']
        },
        'K-Nearest Neighbors': {
            'model__n_neighbors': [3, 5, 7, 10, 15],
            'model__weights': ['uniform', 'distance'],
            'model__p': [1, 2],
            'model__leaf_size': [10, 30, 50]
        },
        'Kernel Ridge Regression': {
            'model__alpha': [0.01, 0.1, 1, 10],
            'model__kernel': ['linear', 'polynomial', 'rbf'],
            'model__degree': [2, 3, 4],
            'model__gamma': [0.01, 0.1, 1, None]
        },
        'Ridge Regression': {
            'model__alpha': [0.01, 0.1, 1, 10, 100, 1000]
        },
        'Lasso Regression': {
            'model__alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10]
        },
        'ElasticNet': {
            'model__alpha': [0.0001, 0.001, 0.01, 0.1, 1],
            'model__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
        }
    }
    
    if model_name not in param_grids:
        print(f"No parameter grid defined for {model_name}")
        return None
    
    # Perform randomized search
    random_search = RandomizedSearchCV(
        pipelines[model_name],
        param_grids[model_name],
        n_iter=50,  # Increased for better search
        scoring='neg_root_mean_squared_error',
        cv=5,  # Increased folds for better validation
        verbose=1,
        n_jobs=-1,
        random_state=42
    )
    
    random_search.fit(X_train, y_train)
    
    print(f"\nBest parameters for {model_name}:")
    for param, value in random_search.best_params_.items():
        print(f"  {param}: {value}")
    
    print(f"\nBest cross-validation RMSE: {-random_search.best_score_:.2f}")
    
    return random_search.best_estimator_

In [29]:
def create_final_model(best_model_name, pipelines, X_train, y_train, X_test, y_test):
    """
    Create and evaluate the final selected model
    """
    print(f"\n{'='*60}")
    print(f"FINAL MODEL: {best_model_name.upper()}")
    print('='*60)
    
    # Get the model pipeline
    final_pipeline = pipelines[best_model_name]
    
    # Optionally perform hyperparameter tuning
    tuned_model = hyperparameter_tuning(pipelines, X_train, y_train, best_model_name)
    
    if tuned_model:
        final_pipeline = tuned_model
    
    # Fit the final model on all training data
    final_pipeline.fit(X_train, y_train)
    
    # Make predictions
    y_pred_train = final_pipeline.predict(X_train)
    y_pred_test = final_pipeline.predict(X_test)
    
    # Calculate metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    
    print(f"\nFinal Model Performance:")
    print(f"  Training RMSE: {train_rmse:.2f}")
    print(f"  Testing RMSE: {test_rmse:.2f}")
    print(f"  Training R²: {train_r2:.3f}")
    print(f"  Testing R²: {test_r2:.3f}")
    
    # Feature importance
    if hasattr(final_pipeline.named_steps['model'], 'feature_importances_'):
        importances = final_pipeline.named_steps['model'].feature_importances_
        if hasattr(final_pipeline.named_steps['preprocessor'], 'get_feature_names_out'):
            feature_names = final_pipeline.named_steps['preprocessor'].get_feature_names_out()
        else:
            feature_names = X_train.columns
        
        feature_importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': importances
        }).sort_values('importance', ascending=False)
        
        print(f"\nTop 10 Most Important Features:")
        print(feature_importance_df.head(10).to_string(index=False))
        
        # Plot feature importance
        import matplotlib.pyplot as plt
        
        plt.figure(figsize=(12, 6))
        plt.barh(feature_importance_df.head(20)['feature'][::-1], 
                feature_importance_df.head(20)['importance'][::-1])
        plt.xlabel('Importance')
        plt.title(f'Feature Importance - {best_model_name}')
        plt.tight_layout()
        plt.show()
    
    return final_pipeline



In [30]:
def predict_new_data(model, new_data, scaler=None):
    """
    Make predictions on new data
    """
    predictions = model.predict(new_data)
    return predictions



In [ ]:
def run_complete_modeling_pipeline(data):
    """
    Run the complete modeling pipeline
    """
    print("STARTING COMPLETE MODELING PIPELINE")
    print("="*60)
    
    # Create modeling pipeline
    X_train, X_test, y_train, y_test, original_y_train, original_y_test, pipelines = create_modeling_pipeline(data)
    
    # Evaluate all models
    results_df, feature_importances = evaluate_models(pipelines, X_train, X_test, y_train, y_test, original_y_train, original_y_test)
    
    # Select best model based on Test RMSE
    best_model_name = results_df.iloc[0]['Model']
    print(f"\n{'='*60}")
    print(f"SELECTED BEST MODEL: {best_model_name}")
    print(f"Test RMSE: {results_df.iloc[0]['Test RMSE']:.2f}")
    print(f"Test R²: {results_df.iloc[0]['Test R²']:.3f}")
    print('='*60)
    
    # Create and save final model
    final_model = create_final_model(best_model_name, pipelines, X_train, y_train, X_test, y_test)
    
    # Return everything for further analysis
    return {
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'results': results_df,
        'feature_importances': feature_importances,
        'final_model': final_model,
        'best_model_name': best_model_name,
        'all_pipelines': pipelines
    }

: 

In [ ]:
# Load preprocessed data and run complete pipeline
data = pd.read_csv('preprocessed_real_estate_high_corr.csv')

print(f"Loaded data shape: {data.shape}")
print(f"Columns: {list(data.columns)}")
print(f"\nStarting model training pipeline...")

# Run the complete pipeline
results = run_complete_modeling_pipeline(data)

print("\n" + "="*60)
print("PIPELINE EXECUTION COMPLETE!")
print("="*60)

Loaded data shape: (354089, 5)
Columns: ['price', 'postcode_mean_price', 'street_mean_price', 'town_city_mean_price', 'district_mean_price']

Starting model training pipeline...
STARTING COMPLETE MODELING PIPELINE
Number of features: 4
Categorical features: 0
Numerical features: 4
Total samples: 354089
MODEL COMPARISON AND EVALUATION

Training Linear Regression...

Linear Regression Performance (original scale):
  Test RMSE: 223704.30
  Test MAE: 24588.65
  Test R²: 0.835
  Cross-validation RMSE (log scale): 0.19 (+/- 0.01)

Training Ridge Regression...

Ridge Regression Performance (original scale):
  Test RMSE: 223703.88
  Test MAE: 24589.54
  Test R²: 0.835
  Cross-validation RMSE (log scale): 0.19 (+/- 0.01)

Training Lasso Regression...

Lasso Regression Performance (original scale):
  Test RMSE: 337239.89
  Test MAE: 55989.38
  Test R²: 0.624
  Cross-validation RMSE (log scale): 0.22 (+/- 0.01)

Training ElasticNet...

ElasticNet Performance (original scale):
  Test RMSE: 314447.

In [ ]:
def plot_prediction_by_feature(X_test, y_true, y_pred, feature_name, 
                              n_bins=10, figsize=(12, 8)):
    """
    Plot predictions grouped by a specific feature
    """
    if feature_name not in X_test.columns:
        print(f"Feature '{feature_name}' not found in X_test")
        return
    
    feature_values = X_test[feature_name]
    
    # Create bins if numeric
    if pd.api.types.is_numeric_dtype(feature_values):
        bins = pd.cut(feature_values, bins=n_bins)
    else:
        bins = feature_values
    
    # Calculate metrics per bin
    results = []
    for bin_name, group_idx in bins.groupby(bins).groups.items():
        group_y_true = y_true.iloc[group_idx] if hasattr(y_true, 'iloc') else y_true[group_idx]
        group_y_pred = y_pred.iloc[group_idx] if hasattr(y_pred, 'iloc') else y_pred[group_idx]
        
        rmse = np.sqrt(mean_squared_error(group_y_true, group_y_pred))
        mae = mean_absolute_error(group_y_true, group_y_pred)
        r2 = r2_score(group_y_true, group_y_pred)
        
        results.append({
            'bin': bin_name,
            'count': len(group_idx),
            'rmse': rmse,
            'mae': mae,
            'r2': r2,
            'avg_true': group_y_true.mean(),
            'avg_pred': group_y_pred.mean()
        })
    
    results_df = pd.DataFrame(results)
    
    # Plot
    fig, axes = plt.subplots(2, 2, figsize=figsize)
    
    # 1. RMSE by feature bin
    axes[0, 0].bar(range(len(results_df)), results_df['rmse'])
    axes[0, 0].set_xlabel(f'{feature_name} Bins')
    axes[0, 0].set_ylabel('RMSE')
    axes[0, 0].set_title(f'RMSE by {feature_name}')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # 2. Sample count by bin
    axes[0, 1].bar(range(len(results_df)), results_df['count'])
    axes[0, 1].set_xlabel(f'{feature_name} Bins')
    axes[0, 1].set_ylabel('Sample Count')
    axes[0, 1].set_title(f'Sample Distribution by {feature_name}')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # 3. Average Actual vs Predicted
    x_pos = range(len(results_df))
    width = 0.35
    axes[1, 0].bar([p - width/2 for p in x_pos], results_df['avg_true'], 
                   width, label='Actual', alpha=0.7)
    axes[1, 0].bar([p + width/2 for p in x_pos], results_df['avg_pred'], 
                   width, label='Predicted', alpha=0.7)
    axes[1, 0].set_xlabel(f'{feature_name} Bins')
    axes[1, 0].set_ylabel('Average Price')
    axes[1, 0].set_title(f'Average Actual vs Predicted by {feature_name}')
    axes[1, 0].legend()
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # 4. R² by bin
    axes[1, 1].bar(range(len(results_df)), results_df['r2'])
    axes[1, 1].set_xlabel(f'{feature_name} Bins')
    axes[1, 1].set_ylabel('R² Score')
    axes[1, 1].set_title(f'R² Score by {feature_name}')
    axes[1, 1].axhline(y=0, color='r', linestyle='--', alpha=0.5)
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.suptitle(f'Prediction Analysis by {feature_name}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return results_df

def plot_error_by_magnitude(y_true, y_pred, n_bins=10):
    """
    Plot error metrics by actual value magnitude
    """
    # Bin by actual value magnitude
    bins = pd.qcut(y_true, q=n_bins, duplicates='drop')
    
    results = []
    for bin_name, group_idx in bins.groupby(bins).groups.items():
        group_y_true = y_true.iloc[group_idx] if hasattr(y_true, 'iloc') else y_true[group_idx]
        group_y_pred = y_pred.iloc[group_idx] if hasattr(y_pred, 'iloc') else y_pred[group_idx]
        
        rmse = np.sqrt(mean_squared_error(group_y_true, group_y_pred))
        mae = mean_absolute_error(group_y_true, group_y_pred)
        mape = np.mean(np.abs((group_y_true - group_y_pred) / group_y_true)) * 100
        
        results.append({
            'price_range': bin_name,
            'avg_price': group_y_true.mean(),
            'rmse': rmse,
            'mae': mae,
            'mape': mape,
            'count': len(group_idx)
        })
    
    results_df = pd.DataFrame(results)
    
    # Plot
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. RMSE vs Price Range
    axes[0, 0].plot(results_df['avg_price'], results_df['rmse'], 'bo-')
    axes[0, 0].set_xlabel('Average Price in Bin')
    axes[0, 0].set_ylabel('RMSE')
    axes[0, 0].set_title('RMSE by Price Range')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. MAE vs Price Range
    axes[0, 1].plot(results_df['avg_price'], results_df['mae'], 'ro-')
    axes[0, 1].set_xlabel('Average Price in Bin')
    axes[0, 1].set_ylabel('MAE')
    axes[0, 1].set_title('MAE by Price Range')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. MAPE vs Price Range
    axes[1, 0].plot(results_df['avg_price'], results_df['mape'], 'go-')
    axes[1, 0].set_xlabel('Average Price in Bin')
    axes[1, 0].set_ylabel('MAPE (%)')
    axes[1, 0].set_title('Mean Absolute Percentage Error by Price Range')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Sample Distribution
    axes[1, 1].bar(range(len(results_df)), results_df['count'])
    axes[1, 1].set_xlabel('Price Range Bin')
    axes[1, 1].set_ylabel('Sample Count')
    axes[1, 1].set_title('Sample Distribution by Price Range')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.suptitle('Error Analysis by Price Magnitude', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return results_df